In [ ]:
from os import chdir, listdir, makedirs
from os.path import exists

import pandas as pd

In [2]:
# Move up two directories, to project base directory
chdir("..\\..")

# Defines output directory
output_dir = "data\\"

# Makes output directory if it doesn't exist
if not exists(output_dir):
    makedirs(output_dir)

# Gets all files in store_info subfolder
paths = listdir('data\\store_info\\')

In [ ]:
# Instantiates empty dataframe to hold store page information
df_store_info = pd.DataFrame()

# Instantiates empty list to hold files that could not be merged successfully
failed_merges = []

# Loops through all files...
for i, path in enumerate(paths):
    try:
        # Attempting to read files...
        df_store_info_current = pd.read_json(f'data\\store_info\\{path}', orient='index')
        
        # Appending last read file to dataframe of all files...
        df_store_info = pd.concat([df_store_info, df_store_info_current.transpose()])
    except:
        # Or appending file name to list of failed merges, if unsuccessful.
        failed_merges.append(path)

# Shows head of dataframe for spot inspection
df_store_info.head().transpose()

In [ ]:
# Simple dtype coercion/cleaning

# Integer
df_store_info['steam_appid'] = df_store_info['steam_appid'].astype(int)
df_store_info['required_age'] = df_store_info['required_age'].astype(int)

# Boolean
df_store_info['is_free'] = df_store_info['is_free'].astype(bool)
df_store_info['controller_support'] = df_store_info['controller_support'] == 'full'
df_store_info['is_demo'] = df_store_info['fullgame'].isna()

# Category (Should be "game" for all)
df_store_info['type'] = df_store_info['type'].astype('category')

# Text - remove html formatting (sometimes preserving line breaks)
df_store_info['detailed_description'] = df_store_info['detailed_description'].str.replace('<br>','\n').str.replace('<((?!>)(?!<).)+>', '', regex=True).str.strip()
df_store_info['about_the_game'] = df_store_info['about_the_game'].str.replace('<((?!>)(?!<).)+>', '', regex=True).str.strip()
df_store_info['supported_languages'] = df_store_info['supported_languages'].str.replace('<((?!>)(?!<).)+>', '', regex=True).str.strip()
df_store_info['drm_notice'] = df_store_info['drm_notice'].str.replace('<br>','\n').str.replace('<((?!>)(?!<).)+>', '', regex=True).str.strip()

In [ ]:
# Extracting relevant values from dictionary fields

# Simple extraction
df_store_info['price'] = df_store_info['price_overview'].apply(pd.Series)['initial'].fillna(0)/100
df_store_info['total_achievements'] = df_store_info['achievements'].apply(pd.Series)['total'].fillna(0).astype(int)
df_store_info['released'] = df_store_info['release_date'].apply(pd.Series)['coming_soon']
df_store_info['release_date'] = pd.to_datetime(df_store_info['release_date'].apply(pd.Series)['date'], format="%b %d, %Y", errors='coerce')
df_store_info['recommendations'] = df_store_info['recommendations'].apply(pd.Series)['total'].fillna(0).astype(int)
df_store_info['metacritic'] = df_store_info['metacritic'].apply(pd.Series)['score']

# Platform requirements - three fields of dicts
for platform in ['pc', 'mac', 'linux']:
    df_reqs_curr = df_store_info[f'{platform}_requirements'].apply(pd.Series)
    
    df_store_info[f'{platform}_requirements_minimum'] = df_reqs_curr['minimum'].str.replace('<br>','\n').str.replace('<((?!>)(?!<).)+>', '', regex=True).str.replace('Minimum:',"").str.strip()
    df_store_info[f'{platform}_requirements_recommended'] = df_reqs_curr['recommended'].str.replace('<br>','\n').str.replace('<((?!>)(?!<).)+>', '', regex=True).str.replace('Recommended:',"").str.strip()

# Platform comaptibility - list of dicts 
df_compatibility = df_store_info['platforms'].apply(pd.Series)
for platform in ['windows','mac','linux']:
    df_store_info['compatible_'+platform] = df_compatibility[platform]

In [ ]:
def encode_categories(df_store_info: pd.DataFrame, target_cols: list):
    """Helper function for generating one-hot encodings for categories stored as a list of dictionaries containing category id and name.

    Args:
        df_store_info (pd.DataFrame): Store information dataframe
        target_cols (list): List of columns to transform. Feature must be a list of dictionaries, where each dictionary corresponds to a subcategory of the category, and contains a key called "description" containing the name of that subcategory.

    Returns:
        pd.DataFrame: Input dataframe with new columns.
    """
    
    # Loops through each of the target columns
    for target_col in target_cols:
        # Extracts lists of subcategory names from target column
        # Uses a gross lambda function to handle empty lists (and to avoid writing a helper function for a helper function)
        df_store_info[target_col + "_names"] = df_store_info[target_col].apply(
            lambda categories: (
                [category_dict["description"] for category_dict in categories]
                if type(categories) == list
                else []
            )
        )
        
        # From series containing per-game lists of subcategories, generates intermediate dataframe 
        #   where each record is a game and each field holds the name of a subcategory to which the game belongs.
        #  Not all fields have values for all games;
        #   A game with 3 subcategories will have three populated fields, and the rest will be NA
        df_categories_per_game =  df_store_info[target_col +  "_names"].apply(pd.Series)

        # Instantiates empty set to hold list of all seen categories
        categories_with_na = set()
        
        # Adds categories to set of categories (enforces uniqueness, except for NA)
        for col in df_categories_per_game:
            for category in df_categories_per_game[col].unique():
                categories_with_na.add(category)
        
        
        # Instantiates empty list to hold names of known categories
        
        # Iterates through set of categories (including NAs), adding each category to list if it's not na
        categories = []
        for category in categories_with_na:
            # Type checking since "NA" is not a string
            if type(category) == str:
                categories.append(category)
        
        # Sorts categories
        categories.sort()
        
        # For each 
        for category in categories:
            df_store_info[target_col + ': ' + category] = df_categories_per_game.isin([category]).sum(axis=1) == 1
    
    return df_store_info

In [ ]:
def encode_categories(df_store_info: pd.DataFrame, category_cols: list):
    """Helper function for generating one-hot encodings for categories stored as a list of dictionaries containing category id and name.

    Args:
        df_store_info (pd.DataFrame): Store information dataframe
        target_cols (list): List of columns to transform. Feature must be a list of dictionaries, where each dictionary corresponds to a subcategory of the category, and contains a key called "description" containing the name of that subcategory.

    Returns:
        pd.DataFrame: Input dataframe with new columns.
    """
    
    # Loops through each of the target columns
    for category in category_cols:
        # Extracts lists of subcategory names from target column
        # Uses a gross lambda function to handle empty lists (and to avoid writing a helper function for a helper function)
        df_store_info[category + "_names"] = df_store_info[category].apply(
            lambda subcategories: (
                [subcategory_dict["description"] for subcategory_dict in subcategories]
                if type(subcategories) == list
                else []
            )
        )
        
        # From series containing per-game lists of subcategories, generates intermediate dataframe 
        #   where each record is a game and each field holds the name of a subcategory to which the game belongs.
        #  Not all fields have values for all games;
        #   A game with 3 subcategories will have three populated fields, and the rest will be NA
        df_subcategories_per_game =  df_store_info[category +  "_names"].apply(pd.Series)

        # Instantiates empty set to hold list of all seen categories
        subcategories_with_na = set()
        
        # Adds categories to set of categories (enforces uniqueness, except for NA)
        for col in df_subcategories_per_game:
            for subcategory in df_subcategories_per_game[col].unique():
                subcategories_with_na.add(subcategory)
        
        
        # Instantiates empty list to hold names of known categories
        
        # Iterates through set of categories (including NAs), adding each category to list if it's not na
        subcategories = []
        for subcategory in subcategories_with_na:
            # Type checking since "NA" is not a string
            if type(subcategory) == str:
                subcategories.append(subcategory)
        
        # Sorts categories
        subcategories.sort()
        
        # For each subcategory, checks whether any of each games' subcategories match that subcategory.
        for subcategory in subcategories:
            df_store_info[category + ': ' + subcategory] = df_subcategories_per_game.isin([subcategory]).sum(axis=1) == 1
    
    return df_store_info

In [ ]:
# One-hot encoding of categorical fields stored as a list of dictionaries
df_store_info = encode_categories(df_store_info, ['categories','genres'])

C:\Users\zacha\AppData\Local\Temp\ipykernel_54680\729626719.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_store_info_merged_test['is_demo'] = df_store_info_merged_test['fullgame'].isna()


In [ ]:
# I wanted to be conservative with what data are removed; we only drop these columns because doing so was necessary to save the table.
cols_to_drop = ["pc_requirements",
                "mac_requirements",
                "linux_requirements",
                "package_groups",
                "ratings"]

for col in cols_to_drop:
    df_store_info = df_store_info.drop(col, axis=1)


In [ ]:
# Writes table to a parquet file
df_store_info.to_parquet(output_dir+"store_info.parquet")